<a href="https://colab.research.google.com/github/Sumit-56/Colab_WorkSpace/blob/main/Complete_TSA_and_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#What is Time Series Data?
Time series data is a collection of observations recorded over a sequence of time. This data is typically ordered chronologically and can be used to analyze trends, seasonality, and other patterns that evolve over time. Examples include stock prices, weather data, and website traffic.

In [ ]:
#import library
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#load the data
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bitcoin_price.csv')
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
df.set_index('Date', inplace=True)
df.head()

#Time series index
 we need to convert the Date column to a datetime format and then set it as the index of the DataFrame.

In [ ]:
#convert 'date' to a datetime and set as index
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d")
df.set_index('Date', inplace=True)
df.head()

In [ ]:
#load the data and set the index
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bitcoin_price.csv', index_col='Date', parse_dates=True)
df1.index

# Data Resampling
Upsample: Increase the frequency (e.g., from daily to hourly). This often involves creating new data points through interpolation.
Downsample: Decrease the frequency (e.g., from daily to monthly or yearly). This typically involves aggregating data over the new, lower frequency intervals (e.g., calculating the mean, sum, or other statistics for each month).
Resampling is a crucial step in time series analysis because it allows you to analyze data at different granularities, align time series with different frequencies, and prepare data for various modeling techniques. In your notebook, you used df.resample('M').mean() to downsample your daily Bitcoin price data to a monthly frequency and calculate the mean for each month

In [ ]:
# Resampling to monthly frequency and calculate the mean closing price
df.resample('ME').mean()

#Exploring data

In [ ]:
# 7 day rolling average of closing price
df['7_day_rolling'] = df['Close'].rolling(window =7).mean()
df[['Close', '7_day_rolling']].loc['2023'].plot()
plt.show()

In [ ]:
#fins out the highest avg month
df.resample('M').mean()['Close'].idxmax()

In [ ]:
#calculate daily return
df['daily_returns'] = df['Close'].pct_change()*100

In [ ]:
#days with moret than 10% change in closing price
df[df['daily_returns']>10].head(3)

#Data  Visualization

In [ ]:
#daily closing priceplot
df['Close'].plot(title="Daily Closing Price")
plt.show()


In [ ]:
#plot the yearly volume
df.resample('YE').sum().plot()
plt.show()

In [ ]:
#plotting closing price and 30 day rolling volume
df['30_day_rolling_vol'] = df['Volume'].rolling(window = 30).mean()
df['30_day_rolling_vol'].plot(legend=True)
ax = df['Close'].plot(secondary_y=True, legend = True)
ax.set_ylabel('Volume')
plt.show()

In [ ]:
#correlation between close and 30 day rolling
df33 = df[['Close','30_day_rolling_vol']].corr()

In [ ]:
df33.plot()

#Data Manipulation

In [ ]:
#missing values
df.isnull().sum()

In [ ]:
#extract time variables
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df.head()